![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/training-with-deep-learning/how-to-use-estimator/how-to-use-estimator.png)

# Local Run Using Pytorch Estimator in Azure ML

In this notebook, we use Azure ML's PyTorch estimator to run our training script locally by using the conda environment created for the tutorial.

In [ ]:
import sys

sys.path.append("scripts")
sys.path.append("scripts/cocoapi/PythonAPI/")

import azureml.core
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.dnn import PyTorch

from dotenv import set_key, get_key, find_dotenv
from utilities import get_auth, download_data

import torch
from scripts.XMLDataset import BuildDataset, get_transform
from scripts.maskrcnn_model import get_model

from PIL import Image, ImageDraw
from IPython.display import display

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

## Download data

We first download the dataset that includes the images of store shelves.

In [ ]:
data_file = "Data.zip"
data_url = ("https://bostondata.blob.core.windows.net/builddata/{}".format(data_file))
download_data(data_file, data_url)

## Initialize workspace
Let's load the existing workspace you created earlier in the Azure ML configuration notebook. 

In [ ]:
ws = Workspace.from_config(auth=get_auth(env_path))
print(ws.name, ws.resource_group, ws.location, sep="\n")

## Create an Azure ML experiment
Let's create an experiment and give it a name. The script runs will be recorded under this experiment in Azure.

In [ ]:
exp = Experiment(workspace=ws, name='torchvision')

## Use a train.py script


In [ ]:
with open("scripts/train.py", "r") as f:
    print(f.read())

## Create A Pytorch Estimator

First, we pick the number of epochs to run the training for.This deliberately has a low default value for the speed of running. In actual application, set this to higher values (i.e. num_epochs = 10)

In [ ]:
num_epochs = 1

In [ ]:
script_params = {
    "--data_path": ".",
    "--workers": 8,
    "--learning_rate": 0.005,
    "--epochs": num_epochs,
    "--anchor_sizes": "16,32,64,128,256,512",
    "--anchor_aspect_ratios": "0.25,0.5,1.0,2.0",
    "--rpn_nms_thresh": 0.5,
    "--box_nms_thresh": 0.3,
    "--box_score_thresh": 0.10,
}

estimator = PyTorch(
    source_directory="./scripts",
    script_params=script_params,
    compute_target="local",
    entry_script="train.py",
    use_docker=False,
    user_managed=True,
    use_gpu=True,
)

Next, we point the python interpreter to the local conda environment built for this tutorial. Azure ML SDK will run the training script using this environment. We also turn off project snapshot upload to the cloud since we have a large dataset in the folder.

In [ ]:
estimator.run_config.environment.python.interpreter_path = ("/data/anaconda/envs/TorchDetectAML/bin/python")
estimator.run_config.history.snapshot_project = False

In [ ]:
run = exp.submit(estimator)
RunDetails(run).show()

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
run.get_file_names()

In [ ]:
run.get_metrics()

Let's now register this first model.

In [ ]:
run.register_model(model_name="torchvision_local_model", model_path="/outputs/model_latest.pth")

## Visualize results

Let's download our model and load it to make predictions on our data.

In [ ]:
run.download_file("outputs/model_latest.pth")

In [ ]:
num_classes = 2
anchor_sizes = "16,32,64,128,256,512"
anchor_aspect_ratios = "0.25,0.5,1.0,2.0"
rpn_nms_threshold = 0.5
box_nms_threshold = 0.3
box_score_threshold = 0.1
num_box_detections = 100

In [ ]:
# Load Mask RCNN model
model = get_model(
    num_classes,
    anchor_sizes,
    anchor_aspect_ratios,
    rpn_nms_threshold,
    box_nms_threshold,
    box_score_threshold,
    num_box_detections,
)

In [ ]:
model_path = "model_latest.pth"
model.load_state_dict(torch.load(model_path))
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
# Use a random subset of the data to visualize predictions on the images.
data_path = "./scripts"
dataset = BuildDataset(data_path, get_transform(train=False))
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[-50:])

In [ ]:
for i in range(len(dataset)):
    img, _ = dataset[i]
    model.eval()
    with torch.no_grad():
        prediction = model([img.to(device)])
    img = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
    preds = prediction[0]["boxes"].cpu().numpy()
    print(prediction[0]["scores"])
    draw = ImageDraw.Draw(img)
    for i in range(len(preds)):
        draw.rectangle(
            ((preds[i][0], preds[i][1]), (preds[i][2], preds[i][3])), outline="red"
        )
    display(img)

In the next notebook, we  will [build a custom docker image and push it to Azure Container Registry](03_BuildDockerImage.ipynb). This image will be used for tunning the hyperparameters of the model on AzureMLCompute.